# BASIN_HOPPING

## Overview
This function provides a robust global optimization method for single-variable functions using the basinhopping algorithm from SciPy. Basinhopping is a stochastic global optimization technique that combines random perturbations with local minimization to escape local minima and search for the global minimum of a scalar function. It is especially useful for non-convex optimization problems where the objective function may have multiple local minima or complex landscapes.

The basinhopping algorithm seeks to solve
```math
\min_x f(x)
```
where $f(x)$ is the objective function. At each iteration, the algorithm performs a random displacement:
```math
x_{\text{trial}} = x_{\text{current}} + \delta
```
where $\delta$ is a random vector (with user-defined step size). A local minimization is then performed starting from $x_{\text{trial}}$ to find:
```math
x_{\text{min}} = \operatorname{argmin}_x f(x)
```
The new minimum is accepted with probability
```math
P = \begin{cases} 1 & \text{if } \Delta f \leq 0 \\ \exp(-\Delta f / T) & \text{if } \Delta f > 0 \end{cases}
```
where $\Delta f = f(x_{\text{min}}) - f(x_{\text{current}})$ and $T$ is the temperature parameter. This approach allows the algorithm to escape local minima by occasionally accepting worse solutions, similar to simulated annealing, and increases the chance of finding the global minimum.

This example function is provided as-is without any representation or warranty of accuracy.

## Usage
To use the `basin_hopping` function in Excel, enter it as a formula in a cell:

```excel
=BASIN_HOPPING(func_expr, x_zero, [niter], [T], [stepsize], [minimizer_method])
```

- `func_expr` (string, required): Math expression as a string which accepts a single scalar argument and returns a scalar value.
- `x_zero` (float, required): Initial guess for the variable.
- `niter` (int, optional): Number of basinhopping iterations. Default is `100`.
- `T` (float, optional): Temperature parameter for the acceptance test. Default is `1.0`.
- `stepsize` (float, optional): Step size for random displacement. Default is `0.5`.
- `minimizer_method` (string, optional): Local minimization method to use (e.g., `L-BFGS-B`, `BFGS`, `Powell`). Default is `L-BFGS-B`.

The function returns a 2D list: [[minimum, x_min]], where the first value is the minimum found and the second value is the location of the minimum.

## Example
Here is an example invocation in Excel:

```excel
=BASIN_HOPPING("x**2 + 10*sin(x)", 0, 200, 1.0, 0.5, "L-BFGS-B")
```
This finds the global minimum of the function `x**2 + 10*sin(x)` starting from `0` with 200 iterations, temperature `1.0`, step size `0.5`, and the `L-BFGS-B` minimizer method.

In [ ]:
import numpy as np
from scipy.optimize import basinhopping as scipy_basinhopping
import math
import re

def basin_hopping(func_expr, x_zero, niter=100, T=1.0, stepsize=0.5, minimizer_method='L-BFGS-B'):
    """
    Finds the global minimum of a single-variable function using the basinhopping algorithm.

    Args:
        func_expr (str): Plain math expression as a string (e.g., 'x**2 + 10*sin(x)').
        x_zero (float): Initial guess. Must be a scalar (float or int).
        niter (int, optional): Number of basinhopping iterations. Default is 100.
        T (float, optional): Temperature parameter. Default is 1.0.
        stepsize (float, optional): Step size for random displacement. Default is 0.5.
        minimizer_method (str, optional): Local minimization method. Default is 'L-BFGS-B'.

    Returns:
        2D list: [[minimum, x_min]] or [[error_message]]

    This example function is provided as-is without any representation or warranty of accuracy.
    """
    try:
        if not isinstance(x_zero, (int, float)):
            return [["x_zero must be a scalar (float or int) for single-variable functions."]]
        if not isinstance(func_expr, str):
            return [["func_expr must be a string representing a math expression."]]
        # Inline _to_lambda logic
        funcs = ['sin', 'cos', 'tan', 'exp', 'sqrt', 'log', 'asin', 'acos', 'atan', 'sinh', 'cosh', 'tanh']
        def repl(m):
            return f"math.{m.group(1)}"
        pattern = r'(?<![\w.])(' + '|'.join(funcs) + r')(?=\s*\()'
        expr = re.sub(pattern, repl, func_expr)
        func_expr_lambda = f"lambda x: {expr}"
        try:
            user_func = eval(func_expr_lambda, {"np": np, "math": math, "sin": math.sin, "cos": math.cos, "exp": math.exp, "sqrt": math.sqrt})
        except Exception as e:
            return [[f"Invalid function expression: {str(e)}"]]
        if not callable(user_func) or user_func.__code__.co_argcount != 1:
            return [["func_expr must be a math expression of a single variable (e.g., 'x**2 + 10*sin(x)')."]]
        def func(x):
            try:
                val = user_func(float(x))
                if isinstance(val, np.ndarray):
                    return float(val.item())
                return float(val)
            except Exception:
                return float('inf')
        x0_val = float(x_zero)
        if not isinstance(niter, int) or niter <= 0:
            return [["niter must be a positive integer."]]
        if not isinstance(T, (int, float)) or T <= 0:
            return [["T must be a positive number."]]
        if not isinstance(stepsize, (int, float)) or stepsize <= 0:
            return [["stepsize must be a positive number."]]
        if not isinstance(minimizer_method, str):
            return [["minimizer_method must be a string."]]
        result = scipy_basinhopping(func, x0_val, niter=niter, T=T, stepsize=stepsize, minimizer_kwargs={"method": minimizer_method})
        x_min = result.x
        min_val = result.fun
        if isinstance(min_val, np.ndarray):
            min_val = float(min_val.item())
        else:
            min_val = float(min_val)
        if isinstance(x_min, np.ndarray):
            x_min_list = x_min.tolist()
        else:
            x_min_list = [float(x_min)]
        return [[min_val] + x_min_list]
    except Exception as e:
        return [[str(e)]]

In [ ]:
%pip install -q ipytest
import ipytest
ipytest.autoconfig()

demo_cases = [
    ["x**2 + 10*sin(x)", -2, 300, 1.0, 0.5, "L-BFGS-B"],
    ["(x+2)**2 + 8*cos(1.5*x)", 1, 200, 1.0, 0.5, "L-BFGS-B"],
    ["0.5*x**2 - 6*sin(2*x)", 3, 250, 1.0, 0.5, "L-BFGS-B"],
    ["exp(-0.1*x**2) * sin(3*x)", 0, 200, 1.0, 0.5, "L-BFGS-B"],
    ["(x**2 - 4)**2 + 5*cos(2*x)", 2, 200, 1.0, 0.5, "L-BFGS-B"]
]

def is_valid_type(val):
    if isinstance(val, (float, bool, str)):
        return True
    if isinstance(val, list):
        return all(isinstance(row, list) and all(isinstance(x, (float, bool, str)) for x in row) for row in val)
    return False

import pytest
@pytest.mark.parametrize("func_expr, x_zero, niter, T, stepsize, minimizer_method", demo_cases)
def test_demo_cases(func_expr, x_zero, niter, T, stepsize, minimizer_method):
    result = basin_hopping(func_expr, x_zero, niter, T, stepsize, minimizer_method)
    assert is_valid_type(result), f"Failed: {func_expr} - Output type is not valid. Got: {type(result)} Value: {result}"

def test_invalid_func_expr():
    result = basin_hopping(123, 0)
    assert result == [["func_expr must be a string representing a math expression."]]

def test_invalid_x_zero():
    result = basin_hopping("x**2 + 10*sin(x)", {"bad": "input"})
    assert result == [["x_zero must be a scalar (float or int) for single-variable functions."]]

ipytest.run('-s')

In [ ]:
import gradio as gr
import matplotlib.pyplot as plt
import io
import base64
import numpy as np
import math
import re

def plot_with_min(func_expr, x_zero, niter, T, stepsize, minimizer_method):
    import re
    funcs = ['sin', 'cos', 'tan', 'exp', 'sqrt', 'log', 'asin', 'acos', 'atan', 'sinh', 'cosh', 'tanh']
    def repl(m):
        return f"math.{m.group(1)}"
    pattern = r'(?<![\w.])(' + '|'.join(funcs) + r')(?=\s*\()'
    expr = re.sub(pattern, repl, func_expr)
    func_expr_lambda = f"lambda x: {expr}"
    result = basin_hopping(func_expr, x_zero, niter, T, stepsize, minimizer_method)
    try:
        user_func = eval(func_expr_lambda, {"np": np, "math": math, "sin": math.sin, "cos": math.cos, "exp": math.exp, "sqrt": math.sqrt})
        x = np.linspace(-10, 10, 400)
        y = [user_func(val) for val in x]
        fig, ax = plt.subplots(figsize=(6, 4))
        ax.plot(x, y, label="f(x)")
        if isinstance(result, list) and len(result) > 0 and isinstance(result[0], list) and len(result[0]) == 2 and all(isinstance(v, (float, int)) for v in result[0]):
            min_val, x_min = result[0]
            ax.plot(x_min, min_val, 'ro', label="Minimum")
            ax.annotate(f"min=({x_min:.3f}, {min_val:.3f})", (x_min, min_val), textcoords="offset points", xytext=(0,10), ha='center', color='red')
        ax.set_xlabel('x')
        ax.set_ylabel('f(x)')
        ax.set_title('Function and Minimum')
        ax.legend()
        buf = io.BytesIO()
        plt.tight_layout()
        plt.savefig(buf, format='png')
        plt.close(fig)
        img_b64 = base64.b64encode(buf.getvalue()).decode('utf-8')
        img_url = f"data:image/png;base64,{img_b64}"
        html = f'<img src="{img_url}" alt="Function plot" style="max-width:100%;height:auto;" />'
    except Exception as e:
        html = f"<div style='color:red'>Plot error: {e}</div>"
    return result, html

demo = gr.Interface(
    fn=plot_with_min,
    inputs=[
        gr.Textbox(label="Function Expression", lines=2, value=demo_cases[0][0]),
        gr.Number(label="Initial Guess (x_zero)", value=demo_cases[0][1]),
        gr.Number(label="Number of Iterations (niter)", value=demo_cases[0][2]),
        gr.Number(label="Temperature (T)", value=demo_cases[0][3]),
        gr.Number(label="Step Size", value=demo_cases[0][4]),
        gr.Textbox(label="Minimizer Method", value=demo_cases[0][5]),
    ],
    outputs=[
        gr.Dataframe(
            label="Result Table or Error Message",
            headers=["Minimum Value", "x_min"],
        ),
        gr.HTML(label="Function Plot with Minimum")
    ],
    examples=demo_cases,
    description="Find the global minimum of a single-variable function using the basin_hopping algorithm. Enter the function as a plain math expression (e.g., 'x**2 + 10*sin(x)'). Standard math functions like sin, cos, exp, etc. are supported.",
    flagging_mode="never",
    fill_width=True,
)
demo.launch()